In [ ]:
! [ ! -z "$COLAB_GPU" ] && pip install torch scikit-learn==0.20.* skorch

# import libraries
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import Add, Input 
from tensorflow.keras.utils import to_categorical

(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# This portion of the code is building a convolutional neural network
# normalize the pixel values
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255.0
x_test = x_test/255.0

#Define input shape
input_shape = (32, 32, 3)
filter_size1 = 512
filter_size2 = 256
filter_size3 = 128
filter_size4 = 64
filter_size5 = 10

# Create three convolutional layers including dropout
imgClassModel = Sequential()
# Define 4 layers with filter size 64
imgClassModel.add(Conv2D(filter_size4, kernel_size=(3,3), padding='same',activation='relu',use_bias=False,input_shape=input_shape))
imgClassModel.add(BatchNormalization())
#imgClassModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
imgClassModel.add(Dropout(0.25))
imgClassModel.add(Conv2D(filter_size4, kernel_size=(3,3), padding='same',activation='relu',use_bias=False))
imgClassModel.add(BatchNormalization())
imgClassModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
imgClassModel.add(Dropout(0.25))

# see the shape of each layer
for layer in imgClassModel.layers:
  print('First block')
  print(layer.output_shape)

# Define 4 layers with filter size 128
imgClassModel.add(Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False))
imgClassModel.add(BatchNormalization())
#imgClassModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
imgClassModel.add(Dropout(0.25))
imgClassModel.add(Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False))
imgClassModel.add(BatchNormalization())
imgClassModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
imgClassModel.add(Dropout(0.25))
# see the shape of each layer
for layer in imgClassModel.layers:
  print('Second block')
  print(layer.output_shape)

# Define 2 layers with filter size 256
imgClassModel.add(Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False))
imgClassModel.add(BatchNormalization())
#imgClassModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
imgClassModel.add(Dropout(0.25))
imgClassModel.add(Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False))
imgClassModel.add(BatchNormalization())
imgClassModel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
imgClassModel.add(Dropout(0.25))
for layer in imgClassModel.layers:
  print('Third block')
  print(layer.output_shape)

# Prepare the network to be "fully connected" by flattening the output
imgClassModel.add(Flatten())

# Define MLP the includes 1 hidden layer, and the output layer
imgClassModel.add(Dense(filter_size4,activation='relu'))
imgClassModel.add(Dropout(0.25))
imgClassModel.add(Dense(filter_size5,activation='softmax'))

# format class labels to be used in categorical cross entropy 
ytrain = to_categorical(y_train)
ytest = to_categorical(y_test)

# compile the CNN
imgClassModel.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# fit the CNN on your training data
imgClassModel.fit(x_train,ytrain,epochs=200,batch_size=32)

# predict on testing data
imgClassModel.predict(x_test)
imgClassModel.evaluate(x_test,ytest)


In [ ]:
# This portion of the code is building a convolutional neural network w/ Resnet

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255.0
x_test = x_test/255.0

#Define input shape
input_shape = (32, 32, 3)
filter_size1 = 512
filter_size2 = 256
filter_size3 = 128
filter_size4 = 64
filter_size5 = 10

# Create three convolutional layers including dropout
inputLayer = Input(shape=input_shape)
#inputBlock = Conv2D(filter_size4, kernel_size=(7,7), padding='same',use_bias=False)(inputLayer)
inputBlock = Conv2D(filter_size4, kernel_size=(1,1), padding='same',use_bias=False)(inputLayer)
inputBlock = BatchNormalization()(inputBlock)

# Define 2 layers with filter size 64
hiddenBlock1 = Conv2D(filter_size4, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(inputBlock)
hiddenBlock1 = BatchNormalization()(hiddenBlock1)
hiddenBlock1 = Conv2D(filter_size4, kernel_size=(3,3), padding='same',use_bias=False)(hiddenBlock1)
hiddenBlock1 = BatchNormalization()(hiddenBlock1)

# Add output of layer from convolution block to Input to form residual 
resnetResult1 = Add()([inputBlock,hiddenBlock1])
resnetResult1 = Activation('relu')(resnetResult1)
resnetResult1 = BatchNormalization()(resnetResult1)
#resnetResult1 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(resnetResult1)


# Define 2 layers with filter size 128
hiddenBlock2 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(resnetResult1)
hiddenBlock2 = BatchNormalization()(hiddenBlock2)
hiddenBlock2 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',use_bias=False)(hiddenBlock2)
hiddenBlock2 = BatchNormalization()(hiddenBlock2)
hiddenBlock2 = Dropout(0.25)(hiddenBlock2)

# Add output of layer from convolution block to Input to form residual 
resnetResult_2 = Conv2D(filter_size3, kernel_size=(1,1), padding='same',use_bias=False)(resnetResult1) # make the same dimensions as filter size 128
resnetResult_2 = Add()([resnetResult_2,hiddenBlock2])
resnetResult2 = Activation('relu')(resnetResult_2)
resnetResult2 = BatchNormalization()(resnetResult2)
resnetResult2 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(resnetResult2) # (16,16,x)
resnetResult2 = Dropout(0.25)(resnetResult2)

# Define 2 layers with filter size 128
hiddenBlock3 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(resnetResult2)
hiddenBlock3 = BatchNormalization()(hiddenBlock3)
hiddenBlock3 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock3)
hiddenBlock3 = BatchNormalization()(hiddenBlock3)
#hiddenBlock3 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(hiddenBlock3)
hiddenBlock3 = Dropout(0.25)(hiddenBlock3)


# Add output of layer from convolution block to Input to form residual 
resnetResult3 = Add()([resnetResult2,hiddenBlock3])
resnetResult3 = Activation('relu')(resnetResult3)
resnetResult3 = BatchNormalization()(resnetResult3)
resnetResult3 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(resnetResult3) # (8,8,x)


# Define 2 layers with filter size 256
hiddenBlock4 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(resnetResult3)
hiddenBlock4 = BatchNormalization()(hiddenBlock4)
hiddenBlock4 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock4)
hiddenBlock4 = BatchNormalization()(hiddenBlock4)
hiddenBlock4 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(hiddenBlock4)
hiddenBlock4 = Dropout(0.25)(hiddenBlock4)

# Add output of layer from convolution block to Input to form residual 
#resnetResult_4 = Conv2D(filter_size2, kernel_size=(1,1), padding='same',use_bias=False)(resnetResult3) # make the same dimensions as filter size 256
#resnetResult_4 = Add()([resnetResult_4,hiddenBlock4])
#resnetResult4 = Activation('relu')(resnetResult_4)
#resnetResult4 = BatchNormalization()(resnetResult4)
#resnetResult4 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(resnetResult4)

# Define 2 layers with filter size 256
hiddenBlock5 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock4)
hiddenBlock5 = BatchNormalization()(hiddenBlock5)
hiddenBlock5 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',use_bias=False)(hiddenBlock5)
hiddenBlock5 = BatchNormalization()(hiddenBlock5)
hiddenBlock5 = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(hiddenBlock5)
hiddenBlock5 = Dropout(0.25)(hiddenBlock5)

# Prepare the network to be "fully connected" by flattening the output
hiddenBlock8 = Flatten()(hiddenBlock5)

# Define MLP the includes 1 hidden layer, and the output layer
hiddenBlock8 = Dense(filter_size4,activation='relu')(hiddenBlock8)
hiddenBlock8 = Dropout(0.25)(hiddenBlock8)
outputLayer = Dense(filter_size5,activation='softmax')(hiddenBlock8)

# format class labels to be used in categorical cross entropy 
ytrain = to_categorical(y_train)
ytest = to_categorical(y_test)

resnetModel = Model(inputs = inputLayer, outputs = outputLayer)

# compile the CNN
resnetModel.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# fit the CNN on your training data
resnetModel.fit(x_train,ytrain,validation_data=(x_test, ytest),epochs=200,batch_size=32)

ypredict = resnetModel.predict(x_test)

#This gives the loss error and test accuracy
test_e = resnetModel.evaluate(x_test,ytest)
print(test_e)

In [ ]:
# This portion of the code is the CNN with Resnet (but the skip connections are removed)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255.0
x_test = x_test/255.0

#Define input shape
input_shape = (32, 32, 3)
filter_size1 = 512
filter_size2 = 256
filter_size3 = 128
filter_size4 = 64
filter_size5 = 10

# Create three convolutional layers including dropout
inputLayer = Input(shape=input_shape)
#inputBlock = Conv2D(filter_size4, kernel_size=(7,7), padding='same',use_bias=False)(inputLayer)
inputBlock = Conv2D(filter_size4, kernel_size=(1,1), padding='same',use_bias=False)(inputLayer)
inputBlock = BatchNormalization()(inputBlock)

# Define 2 layers with filter size 64
hiddenBlock1 = Conv2D(filter_size4, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(inputBlock)
hiddenBlock1 = BatchNormalization()(hiddenBlock1)
hiddenBlock1 = Conv2D(filter_size4, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock1)
hiddenBlock1 = BatchNormalization()(hiddenBlock1)

# Define 2 layers with filter size 128
hiddenBlock2 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock1)
hiddenBlock2 = BatchNormalization()(hiddenBlock2)
hiddenBlock2 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock2)
hiddenBlock2 = BatchNormalization()(hiddenBlock2)
hiddenBlock2 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(hiddenBlock2) # (16,16,x)
hiddenBlock2 = Dropout(0.25)(hiddenBlock2)

# Define 2 layers with filter size 128
hiddenBlock3 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock2)
hiddenBlock3 = BatchNormalization()(hiddenBlock3)
hiddenBlock3 = Conv2D(filter_size3, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock3)
hiddenBlock3 = BatchNormalization()(hiddenBlock3)
hiddenBlock3 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(hiddenBlock3) # (8,8,x)
hiddenBlock3 = Dropout(0.25)(hiddenBlock3)

# Define 2 layers with filter size 256
hiddenBlock4 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock3)
hiddenBlock4 = BatchNormalization()(hiddenBlock4)
hiddenBlock4 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock4)
hiddenBlock4 = BatchNormalization()(hiddenBlock4)
hiddenBlock4 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(hiddenBlock4)
hiddenBlock4 = Dropout(0.25)(hiddenBlock4)

# Define 2 layers with filter size 256
hiddenBlock5 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',activation='relu',use_bias=False)(hiddenBlock4)
hiddenBlock5 = BatchNormalization()(hiddenBlock5)
hiddenBlock5 = Conv2D(filter_size2, kernel_size=(3,3), padding='same',use_bias=False)(hiddenBlock5)
hiddenBlock5 = BatchNormalization()(hiddenBlock5)
hiddenBlock5 = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(hiddenBlock5)
hiddenBlock5 = Dropout(0.25)(hiddenBlock5)

# Prepare the network to be "fully connected" by flattening the output
hiddenBlock8 = Flatten()(hiddenBlock5)

# Define MLP the includes 1 hidden layer, and the output layer
hiddenBlock8 = Dense(filter_size4,activation='relu')(hiddenBlock8)
hiddenBlock8 = Dropout(0.25)(hiddenBlock8)
outputLayer = Dense(filter_size5,activation='softmax')(hiddenBlock8)

# format class labels to be used in categorical cross entropy 
ytrain = to_categorical(y_train)
ytest = to_categorical(y_test)

resnetModel = Model(inputs = inputLayer, outputs = outputLayer)

# compile the CNN
resnetModel.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# fit the CNN on your training data
resnetModel.fit(x_train,ytrain,validation_data=(x_test, ytest),epochs=200,batch_size=32)

ypredict = resnetModel.predict(x_test)

#This gives the loss error and test accuracy
test_e = resnetModel.evaluate(x_test,ytest)
print(test_e)